In [ ]:
# %pip install -U langchain-core
# %pip install -qU "langchain[google-genai]"

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [1]:
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# Get API key from environment
google_api_key = os.getenv("GOOGLE_API_KEY")

if not google_api_key:
    raise ValueError("Google API key not found. Please set it in the .env file.")


from langchain.chat_models import init_chat_model

llm = init_chat_model("gemini-2.5-flash", model_provider="google_genai")

c:\Users\ASUS\anaconda3\envs\testone\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field

tagging_prompt = ChatPromptTemplate.from_template(
    """
Extract the desired information from the following passage.

Only extract the properties mentioned in the 'Classification' function.

Passage:
{input}
"""
)


class Classification(BaseModel):
    sentiment: str = Field(description="The sentiment of the text")
    aggressiveness: int = Field(
        description="How aggressive the text is on a scale from 1 to 10"
    )
    language: str = Field(description="The language the text is written in")


# Structured LLM
structured_llm = llm.with_structured_output(Classification)

In [4]:
inp = "Estoy increiblemente contento de haberte conocido! Creo que seremos muy buenos amigos!"
prompt = tagging_prompt.invoke({"input": inp})
response = structured_llm.invoke(prompt)

response

Classification(sentiment='positive', aggressiveness=1, language='Spanish')

In [6]:
inp = "Estoy muy enojado con vos! Te voy a dar tu merecido!"
prompt = tagging_prompt.invoke({"input": inp})
response = structured_llm.invoke(prompt)

response.model_dump()

{'sentiment': 'angry', 'aggressiveness': 10, 'language': 'Spanish'}

In [7]:
class Classification(BaseModel):
    sentiment: str = Field(..., enum=["happy", "neutral", "sad"])
    aggressiveness: int = Field(
        ...,
        description="describes how aggressive the statement is, the higher the number the more aggressive",
        enum=[1, 2, 3, 4, 5],
    )
    language: str = Field(
        ..., enum=["spanish", "english", "french", "german", "italian"]
    )

In [10]:
from enum import Enum
from pydantic import BaseModel, Field

# Define proper Enums for clarity and validation
class Sentiment(str, Enum):
    happy = "happy"
    neutral = "neutral"
    sad = "sad"

class Language(str, Enum):
    spanish = "spanish"
    english = "english"
    french = "french"
    german = "german"
    italian = "italian"

class Classification(BaseModel):
    sentiment: Sentiment = Field(..., description="The sentiment of the passage.")
    aggressiveness: int = Field(
        ...,
        description="How aggressive the statement is (1 = calm, 5 = very aggressive).",
        ge=1, le=5  # ✅ Enforces range between 1 and 5
    )
    language: Language = Field(..., description="The language detected in the passage.")


In [11]:
inp = "Estoy increiblemente contento de haberte conocido! Creo que seremos muy buenos amigos!"
prompt = tagging_prompt.invoke({"input": inp})
structured_llm.invoke(prompt)

Classification(sentiment='Positive', aggressiveness=1, language='Spanish')

In [12]:
inp = "Estoy muy enojado con vos! Te voy a dar tu merecido!"
prompt = tagging_prompt.invoke({"input": inp})
structured_llm.invoke(prompt)

Classification(sentiment='angry', aggressiveness=10, language='Spanish')

In [13]:
inp = "Weather is ok here, I can go outside without much more than a coat"
prompt = tagging_prompt.invoke({"input": inp})
structured_llm.invoke(prompt)

Classification(sentiment='neutral', aggressiveness=1, language='English')